In [1]:
%matplotlib inline

In [2]:
import tensorflow as tf
import zipfile
import os
import pandas as pd
import numpy as np
import pathlib
import datetime
from collections import Counter
import copy
import matplotlib.pyplot as plt
import time

/home/muhammadayub/anaconda3/envs/py35-2/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Helpful Functions

In [3]:
def cleanURL(url):
    p = pathlib.Path(url)
    path = str(p.as_posix()) 
    return path 


def getDF(loc, sheetname):
    dataframe = pd.read_excel(loc, sheetname)
    #https://stackoverflow.com/questions/40950310/strip-trim-all-strings-of-a-dataframe
    dataframe = dataframe.applymap(lambda x: x.strip() if type(x) is str else x)
    return dataframe

def printNulls(df):
    null_columns = df.columns[df.isnull().any()]
    return df[null_columns].isnull().sum() 


def writeDFToFile(dfs, path_): #dfs is an array of dataframes and their sheet names , path needs to have
    time_ = str(datetime.datetime.now())
    current_date_time = time_[0:time_.index(".")]
    current_date_time = current_date_time.replace(":", "-")
    task4_fileoutput = path_+current_date_time+".xlsx"

    writer = pd.ExcelWriter(task4_fileoutput)
    
    for df_tuple in dfs:  
        df = df_tuple[0]
        sheetName = df_tuple[1]
        df.to_excel(writer, sheetName)
    print("file written to :       " + task4_fileoutput)
    writer.save()
    
def plotImg(img):
    if(type(img) ==type(None)):
        img =outputsDataReal[10000]
    arr = []
    for a in img:
        arr = [a] + arr
    plt.pcolor( arr, cmap = 'gist_ncar' )

    plt.show()    

cwd = os.getcwd()
cwd

'/home/muhammadayub/Desktop/CS230/Notebooks/re'

# Load Data

# Generate the Raw Data for the Neural Network 

In [4]:
CATEGORY_AMT = 34  # model5/model3_12_9__541_160.ckpt

In [5]:
devMiniBatches = np.load( '/home/muhammadayub/Desktop/CS230/models_saved/model6/devMiniBatches.npy')
testMiniBatches = np.load( '/home/muhammadayub/Desktop/CS230/models_saved/model6/testMiniBatches.npy' )
minibatches = np.load( '/home/muhammadayub/Desktop/CS230/models_saved/model6/minibatches.npy' )

In [6]:
devMiniBatches

array([289, 271,  79, 293,  71, 377, 180, 265, 115, 415,  43,  55, 294,
       320, 253, 432, 145,  56, 146, 434, 279,  44, 327, 134, 394, 163])

In [7]:
outputs= np.load('/home/muhammadayub/Desktop/CS230/training_data/minibatches_12_16_3hot_normalized/outputsTotal2.npy', 'r' )
inputs =np.load('/home/muhammadayub/Desktop/CS230/training_data/minibatches_12_16_3hot_normalized/inputsTotal2.npy', 'r'  )

In [10]:
print(len(inputs))
print(len(outputs))
#split the data to minibatches
MINIBATCHES_AMT = 524 # number of minibatches 
inputs = np.split(inputs, MINIBATCHES_AMT )
outputs = np.split(outputs, MINIBATCHES_AMT )      

104800
104800


In [26]:

# print(cp[:2])
# print(outputs[0][:2])

[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]
[[0.   0.   0.   0.   0.   0.33 0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.34 0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   1.   0.  ]]


# Get the model and learn from the data

In [27]:
def create_placeholders(n_H0, n_W0, n_C0, n_y):
    """
    Creates the placeholders for the tensorflow session.
    
    Arguments:
    n_H0 -- scalar, height of an input image
    n_W0 -- scalar, width of an input image
    n_C0 -- scalar, number of channels of the input
    n_y -- scalar, number of classes
        
    Returns:
    X -- placeholder for the data input, of shape [None, n_H0, n_W0, n_C0] and dtype "float"
    Y -- placeholder for the input labels, of shape [None, n_y] and dtype "float"
    """


    X = tf.placeholder(shape =[None, n_H0, n_W0, n_C0], dtype = np.float32, name="X")
    Y = tf.placeholder(shape  =[None, n_y], dtype = np.float32 , name="Y")
    ### END CODE HERE ###
    
    return X, Y

In [28]:
def initialize_parameters():    

    W1 = tf.get_variable("W1", [10, 10, 28, 10], initializer =tf.contrib.layers.xavier_initializer(seed = 0))
    W2 = tf.get_variable("W2", [6, 6, 10, 2], initializer =tf.contrib.layers.xavier_initializer(seed = 0))
    W3 = tf.get_variable("W3", [35,200] , initializer =tf.contrib.layers.xavier_initializer(seed = 0) )

    parameters = {"W1": W1, "W2": W2 , 'W3': 'W3'}
    
    return parameters

In [29]:
def forward_prop(X, params):
    W1 = params['W1']
    W2 = params['W2']    
    W3 = params['W3']
    
    #convolution 
    Z1 = tf.nn.conv2d(X,W1, strides = [1,2,2,1], padding = 'VALID')
    
    #bias added automatically # RELU
    A1 = tf.nn.relu(Z1)
    
    #average pooling -> at this point all features/weights are important to us
    P1 = tf.nn.avg_pool(A1, ksize = [1,3,3,1], strides = [1,1,1,1], padding = 'SAME')

    # convolution 
    Z2 = tf.nn.conv2d(P1,W2, strides = [1,2,2,1], padding = 'VALID')
    
    #RELU
    A2 = tf.nn.relu(Z2)
    
    #max pooling
    P2 = tf.nn.max_pool(A2, ksize = [1,3,3,1], strides = [1,1,1,1], padding = 'VALID')
    
    #flatten
    P2 = tf.contrib.layers.flatten(P2)

    #fully connected
    Z3 = tf.contrib.layers.fully_connected(P2, CATEGORY_AMT, activation_fn = None) #1 for yes/no
    #going to add the softmax directly

    return Z3

In [34]:
# 22:04 - 22:08 Tuesday
# 22:10 - 22:20
#Restoring the model
outputData = outputs
inputData = inputs 

In [49]:
filepath = "/home/muhammadayub/Desktop/CS230/models_saved/model6/model6_12_15__194_60.ckpt" #4/model3_12_9__134_100.ckpt" #model4/model3_12_9__9_80.ckpt"
# dev_test_or_train = 1
# if(dev_test_or_train ==1): #train

# filepathEvalMetrics = "/home/muhammadayub/Desktop/CS230/models_saved/model3/model_12_9__"+str(49)+"_"+str(60)+"_evalMetrics_train.npy"
accuracies= []

print(" Running ", len(minibatches) , " minibatches to get the evaluation metrics") # testMiniBatches  devMiniBatches
output_classes = []
predicted_classes = []

preds_one_hot_ev_value = None
output_one_hot_ev_value =None 


graph = tf.Graph()
with graph.as_default(): 
    with tf.Session(graph=graph) as sess:
        
        #define the graph
        X, Y = create_placeholders(64, 64, 28, 34)
        parameters = initialize_parameters()
        Z3 = forward_prop(X, parameters)
        #optimization 
#         cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = Z3, labels = Y))
#         optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)#1e-4).minimize(cross_entropy)

        #calculating the accuracy of the model 
        softmaxZ3 = tf.nn.softmax(Z3) # size will be 2200, 34
        
        top4 =  tf.nn.top_k(softmaxZ3, 3)     
        top4Indices = top4.indices
        oneHotIndices = tf.one_hot(top4Indices, depth = 34)
        summationOfOneHotFor_NHotEncoding = tf.reduce_sum(oneHotIndices, 1)  # this is the N Hot Encoding 

        
        #equals = tf.equal(summationOfOneHotFor_NHotEncoding, Y) #boolean tensor
        #equalsFloat32 = tf.cast(equals, tf.float32)  # 1 or 0 tensor
        
        summationOfOneHotFor_NHotEncoding = tf.cast(summationOfOneHotFor_NHotEncoding, tf.int32)
        Y = tf.cast(Y, tf.int32)
        
        andOp = tf.bitwise.bitwise_and(summationOfOneHotFor_NHotEncoding, Y)
        andOp = tf.cast(andOp, tf.float32)
        sumVal = tf.reduce_sum(andOp, axis = 1)
        sumValDivided = sumVal/3.0
        
        
        accuracy = tf.reduce_mean(sumValDivided)
        
        
        saver = tf.train.Saver()

        saver.restore(sess , filepath)
        print("Model restored.")
        
        #devMiniBatches 
        #testMiniBatches
        countMinibatch = 0
        for i_  in minibatches:

            output_image64 = outputData[i_]
            inputImage64 = inputData[i_]
            
            abc = np.where(output_image64!= 0)
            cp = np.array(output_image64)
            cp[abc[0], abc[1]] = 1
            
            
            summationOfOneHotFor_NHotEncoding_val, Y_val, andOpVal, accuracy_val  = sess.run([summationOfOneHotFor_NHotEncoding,Y,andOp, accuracy], feed_dict={X: inputImage64, Y: cp})
#             equalsFloat32_val = sess.run([equalsFloat32], feed_dict={X: inputImage64, Y: output_image64})
            
            
#             sumValDivided, accuracy_val, nHotEncoding = sess.run([sumValDivided, accuracy, summationOfOneHotFor_NHotEncoding], feed_dict={X: inputImage64, Y: output_image64})
#             sumValDivided_val,accuracy_val,nHotEncoding = sess.run([sumValDivided, accuracy, summationOfOneHotFor_NHotEncoding], feed_dict={X: inputImage64, Y: output_image64})
            
            accuracies.append(accuracy_val)
            output_classes.append(Y_val)
            predicted_classes.append(summationOfOneHotFor_NHotEncoding_val)
            
            print('Minibatch: ', str(countMinibatch), " ", datetime.datetime.now())
            countMinibatch +=1


 Running  470  minibatches to get the evaluation metrics
INFO:tensorflow:Restoring parameters from /home/muhammadayub/Desktop/CS230/models_saved/model6/model6_12_15__194_60.ckpt
Model restored.
Minibatch:  0   2018-12-16 10:25:17.667160
Minibatch:  1   2018-12-16 10:25:19.287598
Minibatch:  2   2018-12-16 10:25:20.907697
Minibatch:  3   2018-12-16 10:25:22.704461
Minibatch:  4   2018-12-16 10:25:24.233625
Minibatch:  5   2018-12-16 10:25:25.946632
Minibatch:  6   2018-12-16 10:25:27.669638
Minibatch:  7   2018-12-16 10:25:29.323396
Minibatch:  8   2018-12-16 10:25:31.223139
Minibatch:  9   2018-12-16 10:25:32.978946
Minibatch:  10   2018-12-16 10:25:35.065317
Minibatch:  11   2018-12-16 10:25:36.809393
Minibatch:  12   2018-12-16 10:25:38.702144
Minibatch:  13   2018-12-16 10:25:40.542982
Minibatch:  14   2018-12-16 10:25:42.511926
Minibatch:  15   2018-12-16 10:25:44.209970
Minibatch:  16   2018-12-16 10:25:46.048136
Minibatch:  17   2018-12-16 10:25:47.922777
Minibatch:  18   2018-12

Minibatch:  181   2018-12-16 10:30:35.772405
Minibatch:  182   2018-12-16 10:30:37.480794
Minibatch:  183   2018-12-16 10:30:39.352255
Minibatch:  184   2018-12-16 10:30:41.316695
Minibatch:  185   2018-12-16 10:30:42.964292
Minibatch:  186   2018-12-16 10:30:44.543973
Minibatch:  187   2018-12-16 10:30:46.336763
Minibatch:  188   2018-12-16 10:30:48.097493
Minibatch:  189   2018-12-16 10:30:50.044813
Minibatch:  190   2018-12-16 10:30:51.679797
Minibatch:  191   2018-12-16 10:30:53.407716
Minibatch:  192   2018-12-16 10:30:55.077603
Minibatch:  193   2018-12-16 10:30:56.884154
Minibatch:  194   2018-12-16 10:30:58.655449
Minibatch:  195   2018-12-16 10:31:00.412179
Minibatch:  196   2018-12-16 10:31:01.776077
Minibatch:  197   2018-12-16 10:31:03.270762
Minibatch:  198   2018-12-16 10:31:04.595197
Minibatch:  199   2018-12-16 10:31:06.437017
Minibatch:  200   2018-12-16 10:31:07.938106
Minibatch:  201   2018-12-16 10:31:09.784828
Minibatch:  202   2018-12-16 10:31:11.383702
Minibatch:

Minibatch:  364   2018-12-16 10:35:53.400843
Minibatch:  365   2018-12-16 10:35:55.062404
Minibatch:  366   2018-12-16 10:35:56.901270
Minibatch:  367   2018-12-16 10:35:58.501657
Minibatch:  368   2018-12-16 10:36:00.148350
Minibatch:  369   2018-12-16 10:36:02.024594
Minibatch:  370   2018-12-16 10:36:03.865368
Minibatch:  371   2018-12-16 10:36:05.732353
Minibatch:  372   2018-12-16 10:36:07.247004
Minibatch:  373   2018-12-16 10:36:08.831782
Minibatch:  374   2018-12-16 10:36:10.686022
Minibatch:  375   2018-12-16 10:36:12.364538
Minibatch:  376   2018-12-16 10:36:13.843341
Minibatch:  377   2018-12-16 10:36:15.841509
Minibatch:  378   2018-12-16 10:36:17.559151
Minibatch:  379   2018-12-16 10:36:19.418088
Minibatch:  380   2018-12-16 10:36:21.170189
Minibatch:  381   2018-12-16 10:36:23.229630
Minibatch:  382   2018-12-16 10:36:24.835824
Minibatch:  383   2018-12-16 10:36:26.351942
Minibatch:  384   2018-12-16 10:36:28.182814
Minibatch:  385   2018-12-16 10:36:29.987033
Minibatch:

In [50]:
# print(summationOfOneHotFor_NHotEncoding_val[0])
print(Y_val[0])
# print(andOpVal[0])
# type(andOpVal)

# andOpVal.shape
# summationOfOneHotFor_NHotEncoding_val.shape
len(predicted_classes)

[0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]


470

In [51]:
            
#new graph for concatenating things and getting into the right format for the F1 score 
graph = tf.Graph()
with graph.as_default(): 
    with tf.Session(graph=graph) as sess:            
    
        val1 = tf.concat(output_classes, axis = 0)
        
    
        #after running the minibatches-> we can concatenate the values 
        preds = tf.concat(predicted_classes,axis = 0)
        data_outputs = tf.concat(output_classes,axis = 0)
        
        preds_one_hot_ev, output_one_hot_ev = sess.run([preds, data_outputs])
        preds_one_hot_ev_value=preds_one_hot_ev
        output_one_hot_ev_value =output_one_hot_ev
        
#         d = 35 # d is the total number of classes you have 
        #classLabels =  np.array([2,3,5,6,5,4,2,0,0])# the indexes 
        # tf.one_hot(classLabels, d)

#         preds_one_hot = tf.one_hot(preds, d)
#         data_outputs_one_hot =tf.one_hot(data_outputs, d)#evaluated 
#         preds_one_hot_ev, output_one_hot_ev = sess.run([preds_one_hot, data_outputs_one_hot])
        
#         preds_one_hot_ev_value =preds_one_hot_ev
#         output_one_hot_ev_value =output_one_hot_ev
        
#For Debugging below:
#         zVal = sess.run([Z3], feed_dict={X: inputImage64, Y: output_image64})
#         softMaxVal = sess.run([softmaxZ3], feed_dict={X: inputImage64, Y: output_image64})
#         output_class_val = sess.run([output_class], feed_dict={X: inputImage64, Y: output_image64})
#         num_correct_val = sess.run([num_correct], feed_dict={X: inputImage64, Y: output_image64})
#         num_correct_to_int_val = sess.run([num_correct_to_int], feed_dict={X: inputImage64, Y: output_image64})

In [52]:
_y_true_ = None
_y_pred_ = None 
_f1Score_ = None 
_precision_ = None 
_recall_ =None 
TPVal = None
FPVal = None
FNVal = None 
_f1_ = None 
#using the F1 function found online -> get the values for the predictions  
graph = tf.Graph()
with graph.as_default(): 
    y_true = tf.Variable(output_one_hot_ev_value)
    y_pred = tf.Variable(preds_one_hot_ev_value)
    y_true = tf.cast(y_true, tf.float64)
    y_pred = tf.cast(y_pred, tf.float64)

    #moded from -> https://stackoverflow.com/questions/35365007/tensorflow-precision-recall-f1-score-and-confusion-matrix
    TP = tf.count_nonzero(y_pred * y_true, axis=0) 
    FP = tf.count_nonzero(y_pred * (y_true - 1), axis=0) 
    FN = tf.count_nonzero((y_pred - 1) * y_true, axis=0)  # .001 for numerical stability 

    #they are integers right now and should be floats for numerical stability 
    TP = tf.cast(TP, tf.float32)
    FP = tf.cast(FP, tf.float32)
    FN = tf.cast(FN, tf.float32)
    
    
    precision = TP / (TP + FP+tf.constant(.001))
    recall = TP / (TP + FN+tf.constant(.001))
    f1 = 2 * precision * recall / (precision + recall+tf.constant(.001))

    f1Score = tf.reduce_mean(f1)

    
    with tf.Session(graph=graph) as sess:    
        tf.global_variables_initializer().run(session=sess)
        _f1_, _y_true_, _y_pred_, _f1Score_ , _precision_ ,_recall_ , TPVal,FPVal, FNVal = sess.run([f1, y_true, y_pred, f1Score, precision ,recall, TP,FP, FN ])          


In [53]:
sum(np.array(accuracies).flatten())/len(accuracies)*100

37.542553281688946

In [55]:
_f1_
# _precision_
# _recall_ 
# _f1Score_

array([0.        , 0.        , 0.48776957, 0.        , 0.        ,
       0.        , 0.51086456, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.62970567, 0.        ], dtype=float32)

In [117]:
data = np.array([[0. ,  0.34, 0.33 ,0. ,  0.  , 0. ,  0. ,  0.   ,0.  , 0. , 
                     0.   ,0. ,  0.,   0., 0.  , 0. ,  0. ,  0.,  0.   ,0.  , 0.  ,
                     0. ,  0. ,  0. ,  0.  , 0. ,  0.  , 0., 0. ,  0. ,  0. ,
                     0.  , 0.33, 0.  ], 
                [0.27 ,  0., 0. ,0.35 ,  0.38  , 0. ,  0. ,  0.   ,0.  , 0. , 
                     0.   ,0. ,  0.,   0., 0.  , 0. ,  0. ,  0.,  0.   ,0.  , 0.  ,
                     0. ,  0. ,  0. ,  0.  , 0. ,  0.  , 0., 0. ,  0. ,  0. ,
                     0.  , 0., 0.  ]])

graph = tf.Graph()
with graph.as_default(): 
    with tf.Session(graph=graph) as sess:
        x = tf.placeholder(shape = (2,34), dtype=np.float32)
        top4 =  tf.nn.top_k(x, 3)     

        top4Indices = top4.indices
        oneHotIndices = tf.one_hot(top4Indices, depth = 34)
        summationOfOneHotFor_NHotEncoding = tf.reduce_sum(oneHotIndices, 1)
        
#         top4Vals = sess.run([top4], feed_dict={x:data})
# top4Vals[0].indices
        top4IndicesVals = sess.run([top4Indices], feed_dict={x:data})
        
        oneHotIndicesValues = sess.run([oneHotIndices], feed_dict={x:data})
        
        NHotEncodingValues = sess.run([summationOfOneHotFor_NHotEncoding], feed_dict={x:data})
        
        print(oneHotIndicesValues[0].shape)
        print(top4IndicesVals[0].shape)
        
        print(oneHotIndicesValues)
        print(top4IndicesVals)
        
        print('----------------')
        
        print(NHotEncodingValues[0].shape)
        
        print(NHotEncodingValues)
        
        
        

# tf.math.top_k 

(2, 3, 34)
(2, 3)
[array([[[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0.]],

       [[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 

In [24]:
graph = tf.Graph()
with graph.as_default():
    with tf.Session(graph=graph) as sess:
        ab = tf.constant(np.array([[1,2,4],[42,1,5]]), dtype = np.float32)
        bc = tf.constant(np.array([[5,3,4],[42,1,6]]), dtype = np.float32)
        
        equals = tf.equal(ab, bc)
        equalsFloat32 = tf.cast(equals, tf.float32)
        
        sumVal = tf.reduce_sum(equalsFloat32, axis = 1)
        sumValDivided = sumVal/3.0
        
        meanReduced = tf.reduce_mean (sumValDivided )
        
        result = sess.run([meanReduced])
        print(result[0])
        
        
        
        x = tf.constant([[1,2],[3.,4]])
        
        mean = tf.reduce_mean(x, axis = 0)
        print(sess.run([mean]))
        
        

0.5
[array([2., 3.], dtype=float32)]
